In [ ]:
import sys
import os

# Import mitsuba and set the variant
# To set a variant, you need to have set it in the mitsuba.conf file
# https://mitsuba.readthedocs.io/en/latest/src/key_topics/variants.html
import mitsuba as mi
mi.set_variant('llvm_ad_rgb')

import drjit as dr

# Import the mitsuba-transient extension
# adds extra functionality to existing Mitsuba classes
sys.path.insert(1, '../..')
import mitransient as mitr

In [ ]:
# Load XML file
# You can also use mi.load_dict and pass a Python dict object
# but it is probably much easier for your work to use XML files
scene = mi.load_file(os.path.abspath('cornell-box/cbox_diffuse_spad.xml'))

In [ ]:
# Prepare transient integrator for transient path tracing
# Does the work to initialize the scene before path tracing
transient_integrator = scene.integrator()
transient_integrator.prepare_transient(scene, sensor=0)
# sensor=0 just tells it to use the first sensor in the scene
# (in case there is more than one sensor in the XML file)

In [ ]:
# Render the scene and develop the data
# Because mitsuba works with Dr.JIT, this cell will execute
# very fast. But when you actually read the variables
# it will actually start computing everyting (and take a while).
data_steady, data_transient = transient_integrator.render(scene)
dr.eval(data_steady, data_transient)

In [ ]:
# Plot the computed steady image
mi.util.convert_to_bitmap(data_steady)

In [ ]:
import numpy as np

data_max = np.max(data_transient)

mitr.utils.show_video((np.moveaxis(data_transient, 0, 1) / data_max) ** (1.0 / 6.0), 2)
# mitr.utils.save_frames((np.moveaxis(data_transient, 0, 1) / data_max) ** (1.0 / 6.0), 2,'/home/jsolan/video')